<a href="https://colab.research.google.com/github/alessandrobessi/cuda-lab/blob/master/CUDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt update -qq;
!wget https://developer.nvidia.com/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb;
!dpkg -i cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb;
!apt-key add /var/cuda-repo-8-0-local-ga2/7fa2af80.pub;
!apt-get update -qq;
!apt install cuda-8.0;

50 packages can be upgraded. Run 'apt list --upgradable' to see them.
--2019-01-17 06:37:55--  https://developer.nvidia.com/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb
Resolving developer.nvidia.com (developer.nvidia.com)... 192.229.162.216
Connecting to developer.nvidia.com (developer.nvidia.com)|192.229.162.216|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/8.0/secure/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64.deb?AvOLDbgjahHRRssO-U9Lsj78X64bUSQFiz7W4F98EJyeofU16T564fXYFtylUeuKFptUqnjzvUHTiwkEN7Ay40xmVstJpHldF5wtfcuvPIgmSqlbJDzgjELHKeVChqHG9xllhl8ggXB8Q8-f3aUb7JtSzJKYmO0-EnzS-YLiovnP4tvafcLMXhvt9MHX05MyQijh1BbuiQ2SRYiD558xp2wbBg [following]
--2019-01-17 06:37:55--  https://developer.download.nvidia.com/compute/cuda/8.0/secure/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64.deb?AvOLDbgjahHRRssO-U9Lsj

In [7]:
!apt-get install cuda gcc-5 g++-5 -y -qq;
!ln -sf /usr/bin/gcc-5 /usr/local/cuda/bin/gcc;
!ln -sf /usr/bin/g++-5 /usr/local/cuda/bin/g++;

!/usr/local/cuda/bin/nvcc --version
!gcc --version
!g++ --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2016 NVIDIA Corporation
Built on Tue_Jan_10_13:22:03_CST_2017
Cuda compilation tools, release 8.0, V8.0.61
gcc (Ubuntu 5.5.0-12ubuntu1) 5.5.0 20171010
Copyright (C) 2015 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

g++ (Ubuntu 5.5.0-12ubuntu1) 5.5.0 20171010
Copyright (C) 2015 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [26]:
!pip install git+git://github.com/alessandrobessi/nvcc4jupyter.git

  Cloning git://github.com/alessandrobessi/nvcc4jupyter.git to /tmp/pip-req-build-zwggwjmg
  Running setup.py bdist_wheel for NVCCPlugin ... - done
  Stored in directory: /tmp/pip-ephem-wheel-cache-fbabwt5m/wheels/af/b7/9c/ef07a4aaee4995dc4cf4740454641b9b99b6aad7aa2dab2ee2
Successfully built NVCCPlugin


In [27]:
import os
os.environ['PATH'] += ':/usr/local/cuda/bin'
%load_ext nvcc_plugin
%reload_ext nvcc_plugin

The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin
directory /content/src already exists
Out bin /content/result.out


In [28]:
%%cu
#include <stdio.h>

int main() {
    printf("Hello world\n");
    return 0;
}

'Hello world\n'

In [38]:
!/usr/local/cuda/bin/nvprof ./result.out

Hello world
======== Warning: No CUDA application was profiled, exiting


In [49]:
%%cuda -n add.cu
#include <stdlib.h>
#include <stdio.h>
#include <math.h>

/* 
GPU kernel
    threadIdx.x   index of the current thread within its block
    blockDim.x    number of threads in the block
    gridDim.x     number of thread blocks
    blockIdx.x    index of current block within the grid
*/
__global__ void add(int n, float *x, float *y)
{
    int index = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;
    for (int i = index; i < n; i+= stride)
    {
        y[i] = x[i] + y[i];
    }
}

int main()
{
    int N = 1 << 20; // 1M elements

    // allocate unified memory -- accessible from CPU or GPU
    float *x, *y;
    cudaMallocManaged(&x, sizeof(float) * N);
    cudaMallocManaged(&y, sizeof(float) * N);

    // initialize x and y arrays on the host
    for (int i = 0; i < N; i++)
    {
        x[i] = 1.0f;
        y[i] = 2.0f;
    }

    // run kernel on 1M elements on the CPU
    // CUDA GPUs run kernels using blocks of threads that are a multiple of 32 in size
    int blockSize = 256;
    int numBlocks = (N + blockSize - 1) / blockSize;
    add<<<numBlocks, blockSize>>>(N, x, y);

    // wait for GPU to finish before accessing on host
    cudaDeviceSynchronize();

    // check for errors (all values should be 3.0f)
    float maxError = 0.0f;
    for (int i = 0; i < N; i++)
    {
        maxError = fmax(maxError, fabs(y[i] - 3.0f));
    }
    printf("Max error: %f\n", maxError);

    // free memory
    cudaFree(x);
    cudaFree(y);

    return 0;
}

'File written in /content/src/add.cu'

In [51]:
!/usr/local/cuda/bin/nvcc src/add.cu -o add
!/usr/local/cuda/bin/nvprof ./add

nvcc warning : The 'compute_20', 'sm_20', and 'sm_21' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
==2819== NVPROF is profiling process 2819, command: ./add
Max error: 0.000000
==2819== Profiling application: ./add
==2819== Profiling result:
Time(%)      Time     Calls       Avg       Min       Max  Name
100.00%  103.01us         1  103.01us  103.01us  103.01us  add(int, float*, float*)

==2819== Unified Memory profiling result:
Device "Tesla K80 (0)"
   Count  Avg Size  Min Size  Max Size  Total Size  Total Time  Name
       6  1.3333MB  128.00KB  2.0000MB  8.000000MB  1.167648ms  Host To Device
      84  146.29KB  4.0000KB  0.9961MB  12.00000MB  1.869344ms  Device To Host
Total CPU Page faults: 42

==2819== API calls:
Time(%)      Time     Calls       Avg       Min       Max  Name
 97.16%  135.88ms         2  67.940ms  893.74us  134.99ms  cudaMallocManaged
  1.54%  2.1521ms         1  2.1521ms  2.1521ms  2

In [42]:
!ls

cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb  result.out   src
gcc						       sample_data


In [45]:
!ls src

example.cu
